In [1]:
from sklearn.impute import KNNImputer


In [2]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler,QuantileTransformer,KBinsDiscretizer
import glob
import imblearn
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from datetime import date

In [3]:
knn =  KNNImputer(n_neighbors=2)


In [4]:
train=  pd.read_csv("/home/kruzr/personal_git/data/hackeearth/Dataset/train.csv")
test =  pd.read_csv("/home/kruzr/personal_git/data/hackeearth/Dataset/test.csv")

In [5]:
train["condition"] = knn.fit_transform(train[["condition"]])

In [6]:
test["condition"] = knn.transform(test[["condition"]])

In [7]:
train_x_cols = ["pet_id","issue_date","listing_date","condition","color_type","length(m)","height(cm)","X1","X2"]
train_y_cols = ["breed_category","pet_category"]

In [8]:
train_x =  train[train.columns.difference(train_y_cols)]
train_y =  train[train_y_cols]

In [9]:
train_y["breed_category"] = train_y["breed_category"].astype(int)

/home/kruzr/work/virtual_envs/notebookenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
# train_mms_len =  StandardScaler()
# train_mms_he  =  StandardScaler()
train_mms_len =  KBinsDiscretizer(n_bins=10,encode="ordinal")


In [11]:
X1_sub = train.X1.value_counts().reset_index()["index"][:18].to_list()

In [12]:
color_type_sub = train.color_type.value_counts()[:30].reset_index()["index"].to_list()

In [13]:
def feature_engine(df,test=False):
    df["days"]  = (pd.DatetimeIndex(df['listing_date'])-pd.DatetimeIndex(df['issue_date'])).days
    df["x1x2"] = df["X1"]*df["X2"]
    df["x1px2"] = df["X1"]+df["X2"]
    df["x1mx2"] = abs(df["X1"]-df["X2"])
    
    df["lh"] = df["length(m)"]*100*df["height(cm)"]
    df["lhx1x2"] = df["x1x2"]*df["lh"]
    df.loc[~df.color_type.isin(color_type_sub),"color_type"]="Other"
    df.loc[~df.X1.isin(X1_sub),"X1"]="Other"
    df = pd.get_dummies(df, columns=['color_type'])
    df = pd.get_dummies(df, columns=['X1'])
#     df = pd.get_dummies(df, columns=['X2'])
    
    if( not test):
        df[["length(m)","height(cm)","x1x2","lh","x1px2","x1mx2","days","X2"]] = train_mms_len.fit_transform(df[["length(m)","height(cm)","x1x2","lh","x1px2","x1mx2","days","X2"]])
        
    else:
        df[["length(m)","height(cm)","x1x2","lh","x1px2","x1mx2","days","X2"]] = train_mms_len.transform(df[["length(m)","height(cm)","x1x2","lh","x1px2","x1mx2","days","X2"]])
    df = pd.get_dummies(df, columns=['length(m)'])
    df = pd.get_dummies(df, columns=["height(cm)"])
    df = pd.get_dummies(df, columns=["x1x2"])
    df = pd.get_dummies(df, columns=["lh"])
    df = pd.get_dummies(df, columns=["x1px2"])
    df = pd.get_dummies(df, columns=['days'])
        
    return df

In [14]:
fe_train_x = feature_engine(train_x,test=False)

/home/kruzr/work/virtual_envs/notebookenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/kruzr/work/virtual_envs/notebookenv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/kruzr/work/virtual_envs/notebookenv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/kruzr/work/virtual_envs/notebookenv/lib/pyt

In [15]:
fe_train_x

,X2,condition,issue_date,listing_date,pet_id,x1mx2,lhx1x2,color_type_Black,color_type_Black Brindle,color_type_Black Tabby,...,days_0.0,days_1.0,days_2.0,days_3.0,days_4.0,days_5.0,days_6.0,days_7.0,days_8.0,days_9.0
0,3.0,2.00000,2016-07-10 00:00:00,2016-09-21 16:25:00,ANSL_69903,2.0,72820.80,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,3.0,1.00000,2013-11-21 00:00:00,2018-12-27 17:47:00,ANSL_66892,2.0,119536.56,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2.0,0.88339,2014-09-28 00:00:00,2016-10-19 08:24:00,ANSL_69750,3.0,36810.00,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1.0,1.00000,2016-12-31 00:00:00,2019-01-25 18:30:00,ANSL_71623,1.0,0.00,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,2.0,2.00000,2017-09-28 00:00:00,2017-11-19 09:38:00,ANSL_57969,3.0,39816.00,1,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,1.0,2.00000,2017-01-26 00:00:00,2018-03-09 15:35:00,ANSL_51738,1.0,0.00,0,0,0,...,0,0,0,0,0,1,0,0,0,0
18830,2.0,0.88339,2016-06-18 00:00:00,2017-07-09 08:37:00,ANSL_59900,3.0,62415.00,0,0,0,...,0,0,0,0,1,0,0,0,0,0
18831,3.0,0.00000,2010-07-21 00:00:00,2018-08-22 14:27:00,ANSL_53210,2.0,325829.79,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18832,3.0,0.00000,2017-05-12 00:00:00,2018-02-08 14:05:00,ANSL_63468,2.0,288416.70,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [16]:
fe_train_x.columns

Index(['X2', 'condition', 'issue_date', 'listing_date', 'pet_id', 'x1mx2',
       'lhx1x2', 'color_type_Black', 'color_type_Black Brindle',
       'color_type_Black Tabby',
       ...
       'days_0.0', 'days_1.0', 'days_2.0', 'days_3.0', 'days_4.0', 'days_5.0',
       'days_6.0', 'days_7.0', 'days_8.0', 'days_9.0'],
      dtype='object', length=106)

In [17]:
train.color_type

0         Brown Tabby
1               White
2               Brown
3               White
4               Black
             ...     
18829        Tricolor
18830           Brown
18831    Calico Point
18832             Tan
18833           Brown
Name: color_type, Length: 18834, dtype: object

In [18]:
fe_test_x  = feature_engine(test,test = True)

In [19]:
fe_test_x.columns

Index(['pet_id', 'issue_date', 'listing_date', 'condition', 'X2', 'x1mx2',
       'lhx1x2', 'color_type_Black', 'color_type_Black Brindle',
       'color_type_Black Tabby',
       ...
       'days_0.0', 'days_1.0', 'days_2.0', 'days_3.0', 'days_4.0', 'days_5.0',
       'days_6.0', 'days_7.0', 'days_8.0', 'days_9.0'],
      dtype='object', length=105)

In [20]:
missing_list = list(set(fe_train_x.columns)-set(fe_test_x.columns))

In [21]:
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import ExtraTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

In [22]:
model_breed    = XGBClassifier(n_estimators=1500,n_jobs=-1,learning_rate=.05,verbosity=1)
model_pet_cat  = XGBClassifier(n_estimators=1500,n_jobs=-1,learning_rate=.05,verbosity=1)
# from sklearn.svm import SVC
# model_breed    = SVC()
# model_pet_cat  = SVC()
# model_breed    = CatBoostClassifier(iterations=2000,learning_rate=.01)
# model_pet_cat  = CatBoostClassifier(iterations=2000,learning_rate=.01)
# model_breed    = XGBClassifier(n_estimators=1500,n_jobs=-1,class_weight="balanced_subsample")
# model_pet_cat  = XGBClassifier(n_estimators=1500,n_jobs=-1,class_weight="balanced_subsample")
# model_breed  = ExtraTreeClassifier()
# model_pet_cat =  ExtraTreeClassifier()
# model_breed    = LogisticRegression()
# model_pet_cat  = LogisticRegression()

In [23]:
use_cols = fe_train_x.columns.difference(["issue_date","listing_date","pet_id"])

for col in missing_list:
    fe_test_x[col] = 0

In [24]:
model_breed.fit(fe_train_x[use_cols].values,train_y["breed_category"].values)
model_pet_cat.fit(fe_train_x[use_cols].values,train_y["pet_category"].values)

XGBClassifier(learning_rate=0.05, n_estimators=1500, n_jobs=-1,
              objective='multi:softprob')

In [26]:
submission_df = pd.DataFrame()
submission_df["pet_id"] =  fe_test_x["pet_id"]
submission_df["breed_category"] = model_breed.predict(fe_test_x[use_cols].values)
submission_df["pet_category"]   = model_pet_cat.predict(fe_test_x[use_cols].values)

In [27]:
submission_df.to_csv("submission_xgb_ran3.csv",index=False)
#35 best

# Inference

In [ ]:
pd.crosstab(submission_df["pet_category"],submission_df["breed_category"])

In [67]:
def ensemble_submissions():
#     sub_list = glob.glob("*.csv")
#     print(sub_list)
    sub_list  = ["ensemble_ensemble_fold_breed.csv","ensemble_breed.csv",]
    list_df = [] 
    for sub in sub_list:
        df =  pd.read_csv(sub)
        list_df.append(df)
    final_df =  pd.DataFrame()
    final_df["pet_id"] = list_df[0]["pet_id"]
    ensemble_submission = list_df[0]
    for sub in list_df[1:]:
        ensemble_submission =  pd.merge(ensemble_submission,sub,on="pet_id")
    column_list         = ensemble_submission.columns.to_list()
    pet_catergory_list  = [x for x in column_list if((x.split("_")[0]=="pet")&(x.split("_")[-1]!="id"))]
    breed_category_list  = [x for x in column_list if((x.split("_")[0]=="breed"))]
    final_df["breed_category"] = ensemble_submission[breed_category_list].mode(axis=1)[0].values
    final_df["pet_category"]   = ensemble_submission[pet_catergory_list].mode(axis=1)[0].values
    return final_df


In [68]:
fdf = ensemble_submissions()

In [69]:
fdf.to_csv("ensemble_breed_1.csv",index=False)
#best ensemble_ensemble_fold_breed

In [28]:
imp_df = pd.DataFrame({"columns":fe_train_x.columns.difference(["pet_id","issue_date","listing_date"]),"impor_br":model_breed.feature_importances_,"impor_pc":model_pet_cat.feature_importances_})

In [29]:
pcr = imp_df.sort_values("impor_pc",ascending=False)

In [30]:
pcr

,columns,impor_br,impor_pc
99,x1x2_0.0,0.000000,0.219039
6,X1_15,0.270706,0.111307
29,color_type_Brown Tabby,0.000992,0.099132
40,color_type_Orange Tabby,0.006730,0.053924
45,color_type_Tan,0.003483,0.045733
...,...,...,...
94,x1px2_0.0,0.000000,0.000000
10,X1_19,0.000000,0.000000
13,X1_5,0.000000,0.000000
2,X1_10,0.000106,0.000000


In [31]:
feature_pc = pcr["columns"].to_list()[:75]

In [ ]:
feature_br

In [32]:
br = imp_df.sort_values("impor_br",ascending=False)

In [33]:
br

,columns,impor_br,impor_pc
51,condition,0.470366,0.005582
6,X1_15,0.270706,0.111307
9,X1_18,0.035480,0.002090
52,days_0.0,0.023712,0.014109
47,color_type_Tortie,0.015149,0.020275
...,...,...,...
13,X1_5,0.000000,0.000000
94,x1px2_0.0,0.000000,0.000000
4,X1_12,0.000000,0.001750
10,X1_19,0.000000,0.000000


In [34]:
feature_br = br["columns"].to_list()[:100]

In [ ]:
feature_br

In [ ]:
imp_df

In [ ]:
#convert continous features to bins
#try catboost
#try kfold CV and train multiple models
#try oversampled model small class pc
#prepare documentation

In [ ]:
train.pet_category.value_counts()

In [56]:
from imblearn.over_sampling import SMOTE

In [57]:
sm = SMOTE()

In [ ]:
print(train_x_cols )

In [ ]:
train_x,train_y = sm.fit_resample(fe_train_x[use_cols].values,train["pet_category"].values)

In [ ]:
list(train_y).count(0),list(train_y).count(4),list(train_y).count(2),list(train_y).count(1)

In [ ]:
xgb_oversample =  XGBClassifier(n_estimators=300,n_jobs=-1,learning_rate=.05,verbosity=1)

In [ ]:
xgb_oversample.fit(train_x,train_y)

In [73]:
sub =  pd.read_csv("ensemble_ensemble_fold_breed.csv")

In [ ]:
sub["pet_category"] = xgb_oversample.predict(fe_test_x[use_cols].values)

In [ ]:
sub.to_csv("undersample_pet.csv",index=False)

In [38]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [39]:
skf =  StratifiedKFold(n_splits=5)

In [40]:
from sklearn.metrics import classification_report

In [70]:
X = fe_train_x[use_cols].values

In [71]:
y = train["pet_category"].values

In [72]:
model_list  = []
i= 1
for train_ix, test_ix in skf.split(X, y):
    print("Running Process for Fold {}".format(i))
    i+=1
    train_X, test_X = X[train_ix], X[test_ix]
    train_y, test_y = y[train_ix], y[test_ix]
    model_1 = XGBClassifier(n_estimators=1500,n_jobs=-1,learning_rate=.03,verbosity=1)
    train_X,train_y = sm.fit_resample(train_X,train_y)
    model_1.fit(train_X,train_y)
    model_list.append(model_1)    
    print(classification_report(test_y,model_1.predict(test_X)))

Running Process for Fold 1
              precision    recall  f1-score   support

           0       0.29      0.24      0.26        17
           1       0.90      0.85      0.87      1437
           2       0.89      0.94      0.91      2124
           4       0.93      0.74      0.82       189

    accuracy                           0.89      3767
   macro avg       0.75      0.69      0.72      3767
weighted avg       0.89      0.89      0.89      3767

Running Process for Fold 2
              precision    recall  f1-score   support

           0       0.38      0.44      0.41        18
           1       0.90      0.85      0.87      1437
           2       0.89      0.93      0.91      2124
           4       0.92      0.81      0.86       188

    accuracy                           0.89      3767
   macro avg       0.77      0.76      0.76      3767
weighted avg       0.89      0.89      0.89      3767

Running Process for Fold 3
              precision    recall  f1-score   sup

In [44]:
 for idx,model in enumerate(model_list):
        sub["pet_category"] =  model.predict(fe_test_x[feature_pc].values)
        sub.to_csv("Submission_fold_pet_exp_{}.csv".format(idx),index=False)

In [ ]:
model_pet_cat.feature_importances_